<a href="https://colab.research.google.com/github/abstractguy/lstm_autoencoder_classifier/blob/master/LSTM_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset: LSTM Autoencoder for Rare Event Binary Classification in Multivariate Time Series**

https://arxiv.org/abs/1809.10717 (please cite this article, if using the dataset).

https://towardsdatascience.com/extreme-rare-event-classification-using-autoencoders-in-keras-a565b386f098

https://github.com/cran2367/autoencoder_classifier/blob/master/autoencoder_classifier.ipynb

In [0]:
# Money maker by Samuel Duclos.
AI = 'LSTM_autoencoder'

In [0]:
# Hardcode parameters.
if AI == 'LSTM':
    main_ticker = 'AAPL'
    trends = False
    batch_size = 64
    epochs = 100
    learning_rate = 0.000003
    test_size = 0.2
    lookback = 5 # Days of past data.
    tickers_list = []

elif AI == 'LSTM_autoencoder':
    main_ticker = 'FB'
    trends = True
    batch_size = 32
    epochs = 100
    learning_rate = 0.0001
    random_seed = 123
    test_size = 0.15
    lookback = 5 # Days of past data.
    gain = 0.055 # Percentage of gain to consider good for trade.
    tickers_list = ['GE', 
                    'ADS', 
                    'INTC', 
                    'AAPL', 
                    'NVDA', 
                    'CSCO', 
                    'AMD', 
                    'AMZN', 
                    'GOOG', 
                    'MSFT', 
                    'S', 
                    'BAC', 
                    'XLNX', 
                    'WFC', 
                    '^DJI', 
                    '^GSPC', 
                    '^NYA', 
                    '^IXIC']

In [0]:
!pip install pytrends

In [0]:
%matplotlib inline

from collections import OrderedDict
from math import sqrt
from os import chdir
from os.path import exists
from datetime import datetime
from tqdm import tqdm
from numpy.random import seed
from numpy import append, array, concatenate, count_nonzero, empty, empty_like, expand_dims, mean, nan, power, var, where, zeros
from pandas import bdate_range, concat, DataFrame, date_range, read_csv, Series
from pandas_datareader.data import DataReader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import auc, classification_report, confusion_matrix, f1_score, mean_squared_error, precision_recall_curve, precision_recall_fscore_support, recall_score, roc_curve
from tensorflow import set_random_seed
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import load_model, Model, Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TerminateOnNaN
from keras.optimizers import Adam
from keras.utils import plot_model
from keras import optimizers, Sequential
from google.colab.drive import mount
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
from pytrends.request import TrendReq
from pylab import rcParams
from seaborn import heatmap

import matplotlib.pyplot as plt

seed(7)
set_random_seed(11)
rcParams['figure.figsize'] = 8, 6
labels = ['Normal', 'Break']

In [0]:
path = '/content/gdrive/'
mount(path)
path = path + 'My Drive/LSTM_autoencoder/'
chdir(path)

In [0]:
def add_ticker_to_tickers(symbol, ticker, tickers, join='inner'):
    tickers = tickers[tickers.index >= ticker.index.min()]

    if symbol not in tickers.columns:
        return concat([tickers, ticker.to_frame(name=symbol)], axis='columns', join=join)
    else:
        return tickers

'''
def add_ticker_to_tickers_old(symbol, ticker, tickers):
    minimum_date = ticker.index.min()
    overall_minimum_date = tickers.index.min()
    if minimum_date > overall_minimum_date:
        start = overall_minimum_date
        tickers = tickers[tickers.index >= minimum_date]

    tickers[symbol][tickers.index.isin(ticker.index)] = ticker

    return tickers
'''

def download_ticker_from_yahoo(symbol, start=None, end=None):
    if start is None or end is None:
        start = datetime(1970, 1, 1)
        end = datetime.now()

    return DataReader(symbol, 'yahoo', start=start, end=end)

def download_trends_from_google(symbol, start=None, end=None):
    if start is None or end is None:
        start = datetime(1970, 1, 1)
        end = datetime.now()

    pytrends.build_payload([symbol], timeframe='today 5-y')
    return pytrends.interest_over_time()[symbol]

def get_stock(main_ticker, tickers_list, path=path, trends=True):
    csv_path = path + 'dataset.csv'

    if exists(csv_path):
        tickers = read_csv(csv_path)
    else:
        start = datetime(1970, 1, 1)
        end = datetime.now()
        tickers = DataReader(main_ticker, 'yahoo', start=start, end=end).High.dropna().to_frame(name=main_ticker)

        for symbol in tqdm(tickers_list, unit='symbol'):
            try:
                #ticker = DataReader(symbol, 'yahoo', start=start, end=end)
                ticker = download_ticker_from_yahoo(symbol, start=start, end=end)
                tickers = add_ticker_to_tickers(symbol, ticker.High, tickers, join='inner')

            except:
                pass

        business_days = (~tickers.dropna().asfreq('D').isna().any(axis='columns')).astype(int)

        if trends:
            pytrends = TrendReq(hl='en-US', tz=360)
            for symbol in tqdm([main_ticker] + tickers_list, unit='symbol'):
                try:
                    #pytrends.build_payload([symbol], timeframe='today 5-y')
                    #ticker = pytrends.interest_over_time()[symbol]
                    ticker = download_trends_from_google(symbol, start=start, end=end)
                    tickers = add_ticker_to_tickers(symbol + '_trend', ticker, tickers, join='outer')

                except:
                    pass

        ticker = DataReader(main_ticker, 'yahoo', start=start, end=end)
        ticker_y = ticker.Close / ticker.Open
        tickers = add_ticker_to_tickers(main_ticker + '_y', ticker_y, tickers, join='inner')

        tickers = tickers.asfreq('D').fillna(method='backfill')
        tickers = add_ticker_to_tickers('Business_days', business_days, tickers, join='inner')

        tickers.to_csv(csv_path)

    return tickers

def extract_column(dataset, column_name):
    extracted = dataset[column_name].copy()
    dataset = dataset.drop(columns=[column_name])
    return dataset, extracted

def delta_time_series(data):
    data = data.astype('float32').values
    return data[1:] - data[:-1]

def plot_dataset(dataset):
    plt.plot(dataset)
    plt.xlabel('Days')
    plt.ylabel('Derivatives')
    plt.show()

def get_y_from_generator(generator):
    '''Get all targets y from a TimeseriesGenerator instance.'''
    y = None

    for i in range(len(generator)):
        batch_y = generator[i][1]

        if y is None:
            y = batch_y
        else:
            y = append(y, batch_y)

    y = y.reshape((-1, 1))
    print(y.shape)
    return y

def binary_accuracy(a, b, name='training'):
    '''Helper function to compute the match score of two binary numpy arrays.'''
    a = a[:,0] > 0
    b = b[:,0] > 0
    assert len(a) == len(b)
    print('Binary accuracy (' + name + ' data):', (a == b).sum() / len(a))

sign = lambda x: (1, -1)[x < 0]

def curve_shift(dataset, shift_by):
    vector = dataset.y.copy()
    for s in range(abs(shift_by)):
        vector += vector.shift(sign(shift_by), fill_value=0)

    dataset['ytmp'] = vector
    dataset = dataset.drop(dataset[dataset['y'] == 1].index)
    dataset = dataset.drop('y', axis=1).rename(columns={'ytmp': 'y'})
    dataset['y'] = (dataset.y > 0).astype(int)

    return dataset

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)],:]) # Gather past records up to lookback.

        output_X.append(t)
        output_y.append(y[i + lookback + 1])

    return output_X, output_y

# 3D -> 2D.
# (samples, timesteps, features) -> (samples, features).
def flatten(X):
    flattened_X = empty((X.shape[0], X.shape[2])) # Sample X feature array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i,X.shape[1] - 1,:]

    return(flattened_X)

# Scale samples individually.
# (samples, timesteps, features) -> (samples, timesteps, features).
def scale(X, scaler):
    for i in range(X.shape[0]):
        X[i,:,:] = scaler.transform(X[i,:,:])

    return X

In [0]:
dataset_csv = get_stock(main_ticker, tickers_list, trends=trends)

In [0]:
dataset, ticker_y = extract_column(dataset_csv, main_ticker + '_y')
dataset, business_days = extract_column(dataset, 'Business_days')
yesterday_date = dataset.index[-1]
yesterday_score = dataset.iloc[-1,dataset.columns == main_ticker]
dataset['Business_days'] = business_days
plot_dataset(dataset)
dataset.shape

In [0]:
plot_dataset(ticker_y)
ticker_y.size

In [0]:
# Ground truth.
dataset['y'] = (ticker_y > (1.0 + (gain / 2.0))).astype(int)
print('Percentage of ones (keep less than 5%):', 
      count_nonzero(dataset.y) / ticker_y.size)

In [0]:
dataset = dataset.reset_index()

In [0]:
print('Before shifting.')
dataset

In [0]:
print('After shifting.')
dataset = curve_shift(dataset, shift_by=-1)
dataset

In [0]:
dataset.index

In [0]:
dataset = dataset.drop(columns=['Date'])

In [0]:
# Converts the DataFrame to a numpy array.
input_X, input_y = extract_column(dataset, 'y')
input_X = input_X.values
input_y = input_y.values

# Number of features.
n_features = input_X.shape[1]

In [0]:
# Test: The 3D tensors (arrays) for LSTM are forming correctly.
print('First instance of y = 1 in the original data.')
display(dataset.iloc[(where(array(input_y) == 1)[0][0] - lookback):(where(array(input_y) == 1)[0][0] + 1),])

# Temporalize the data.
X, y = temporalize(X=input_X, y=input_y, lookback=lookback)

print('For the same instance of y=1, we are keeping past 5 samples in the 3D predictor array, X.')
display(DataFrame(concatenate(X[where(array(y) == 1)[0][0]], axis=0)))

The two tables are the same. This testifies that we are correctly taking 5 samples (= lookback), X(t):X(t-5) to predict y(t).

In [0]:
X_train, X_test, y_train, y_test = train_test_split(array(X), array(y), test_size=test_size, random_state=random_seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_size, random_state=random_seed)

In [0]:
X_train_y0 = X_train[y_train==0]
X_train_y1 = X_train[y_train==1]
X_valid_y0 = X_valid[y_valid==0]
X_valid_y1 = X_valid[y_valid==1]

In [0]:
X_train.shape

In [0]:
X_train_y0.shape

In [0]:
# (sample, channel, lookback, feature) -> (sample, lookback, feature).
X_train = X_train.reshape(X_train.shape[0], lookback, n_features)
X_test = X_test.reshape(X_test.shape[0], lookback, n_features)
X_valid = X_valid.reshape(X_valid.shape[0], lookback, n_features)

X_train_y0 = X_train_y0.reshape(X_train_y0.shape[0], lookback, n_features)
X_train_y1 = X_train_y1.reshape(X_train_y1.shape[0], lookback, n_features)

X_valid_y0 = X_valid_y0.reshape(X_valid_y0.shape[0], lookback, n_features)
X_valid_y1 = X_valid_y1.reshape(X_valid_y1.shape[0], lookback, n_features)

In [0]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(X_train_y0))
X_train_y0_scaled = scale(X_train_y0, scaler)
X_train_y1_scaled = scale(X_train_y1, scaler)
X_train_scaled = scale(X_train, scaler)

In [0]:
# Test scaling validity.
a = flatten(X_train_y0_scaled)
print('Column-wise mean (should be all zeros):', mean(a, axis=0).round(6))
print('Column-wise variance (should be all ones):', var(a, axis=0))

In [0]:
# Scale test and validation sets.
X_valid_scaled = scale(X_valid, scaler)
X_valid_y0_scaled = scale(X_valid_y0, scaler)
X_test_scaled = scale(X_test, scaler)

In [0]:
_, timesteps, n_features = X_train_y0_scaled.shape

In [0]:
LSTM_autoencoder = Sequential()
# Encoder.
LSTM_autoencoder.add(LSTM(units=32, 
                          activation='relu', 
                          input_shape=(timesteps, n_features), 
                          return_sequences=True))

LSTM_autoencoder.add(LSTM(units=16, activation='relu', return_sequences=False))
LSTM_autoencoder.add(RepeatVector(timesteps))
# Decoder.
LSTM_autoencoder.add(LSTM(units=16, activation='relu', return_sequences=True))
LSTM_autoencoder.add(LSTM(units=32, activation='relu', return_sequences=True))
LSTM_autoencoder.add(TimeDistributed(Dense(n_features)))

# Keep parameters less than non-droped out features.
LSTM_autoencoder.summary()

In [0]:
# LSTM autoencoder model for rare stock event prediction.
# Path to model weights (saved periodically).
filepath = path + 'LSTM_autoencoder.h5'
if exists(filepath):
    LSTM_autoencoder = load_model(filepath)
else:
    # Gradient descent optimization.
    optimizer = Adam(lr=learning_rate, clipnorm=1., clipvalue=0.5)

    # Training configuration.
    LSTM_autoencoder.compile(loss='mean_squared_error', optimizer=optimizer)

    # Save model weights after each epoch if validation loss decreased.
    checkpointer = ModelCheckpoint(filepath=filepath, 
                                   save_best_only=True, 
                                   verbose=1)

    # Control learning rate schedule when validation is not improving.
    reduce_lr = ReduceLROnPlateau(factor=0.1, 
                                  patience=epochs // 25, 
                                  verbose=1, 
                                  min_lr=learning_rate / 1000)

    # Various graphics.
    tbc = TensorBoardColab()

    # Shouldn't happen.
    term_on_NaN = TerminateOnNaN()

    history = LSTM_autoencoder.fit(X_train_y0_scaled, 
                                   X_train_y0_scaled, 
                                   epochs=epochs, 
                                   batch_size=batch_size, 
                                   validation_data=(X_valid_y0_scaled, 
                                                    X_valid_y0_scaled), 
                                   callbacks=[checkpointer, 
                                              reduce_lr, 
                                              TensorBoardColabCallback(tbc), 
                                              term_on_NaN], 
                                              verbose=1).history

In [0]:
# Print training and test loss histories.
try:
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])

# Skip because model was probably loaded from file.
except:
    pass

In [0]:
print('Reconstruction error should be less than one.')

train_x_predictions = LSTM_autoencoder.predict(X_train_scaled)
mse = mean(power(flatten(X_train_scaled) - flatten(train_x_predictions), 2), 
           axis=1)

error = DataFrame({'Reconstruction_error': mse, 'True_class': y_train.tolist()})

groups = error.groupby('True_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, 
            group.Reconstruction_error, 
            marker='o', 
            ms=3.5, 
            linestyle='',
            label = 'Break' if name == 1 else 'Normal')

ax.legend()
plt.title('Reconstruction error for different classes')
plt.ylabel('Reconstruction error')
plt.xlabel('Data point index')
plt.show()

In [0]:
# Predictions on validation data.
valid_x_predictions = LSTM_autoencoder.predict(X_valid_scaled)
mse = mean(power(flatten(X_valid_scaled) - flatten(valid_x_predictions), 2), 
           axis=1)

error = DataFrame({'Reconstruction_error': mse, 'True_class': y_valid.tolist()})

precision_rt, recall_rt, threshold_rt = precision_recall_curve(error.True_class, 
                                                               error.Reconstruction_error)

plt.plot(threshold_rt, precision_rt[1:], label='Precision', linewidth=5)
plt.plot(threshold_rt, recall_rt[1:], label='Recall', linewidth=5)
plt.title('Precision and recall for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
plt.legend()
plt.show()

In [0]:
# Predictions on testing data.
test_x_predictions = LSTM_autoencoder.predict(X_test_scaled)
mse = mean(power(flatten(X_test_scaled) - flatten(test_x_predictions), 2), 
           axis=1)

error = DataFrame({'Reconstruction_error': mse, 'True_class': y_test.tolist()})

threshold_fixed = 0.3
groups = error.groupby('True_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, 
            group.Reconstruction_error, 
            marker='o', 
            ms=3.5, 
            linestyle='',
            label= 'Break' if name == 1 else 'Normal')

ax.hlines(threshold_fixed, 
          ax.get_xlim()[0], 
          ax.get_xlim()[1], 
          colors='r', 
          zorder=100, 
          label='Threshold')

ax.legend()
plt.title('Reconstruction error for different classes')
plt.ylabel('Reconstruction error')
plt.xlabel('Data point index')
plt.show()

In [0]:
pred_y = [1 if e > threshold_fixed else 0 for e in error.Reconstruction_error.values]
conf_matrix = confusion_matrix(error.True_class, pred_y)

In [0]:
plt.figure(figsize=(6, 6))

heatmap(conf_matrix, 
        xticklabels=labels, 
        yticklabels=labels, 
        annot=True, 
        fmt='d')

plt.title('Confusion matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [0]:
false_pos_rate, true_pos_rate, thresholds = roc_curve(error.True_class, 
                                                      error.Reconstruction_error)
roc_auc = auc(false_pos_rate, true_pos_rate,)

In [0]:
plt.plot(false_pos_rate, 
         true_pos_rate, 
         linewidth=5, 
         label='AUC = ' + str(round(roc_auc, 3)))
plt.plot([0, 1], [0, 1], linewidth=5)

plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()